# This notebook to test Saadi's conversational agent's features

## Check conversational agent graph

In [ ]:
from IPython.display import Image, display # type: ignore
from conv_agent import graph

display(Image(graph.get_graph().draw_mermaid_png()))

## Create a LangGraph assistant

Instantiate the graph using a configuration (makes it an Assistant).

In [ ]:
# Connect to the LangGraph server and create an assistant
from langgraph_sdk import get_client # type: ignore

client = get_client(url="http://localhost:2024")

assistant = await client.assistants.create(
    # graph name from langgraph.json
    graph_id="conv_agent",
    config={
        "recursion_limit": 1000000000,
        "configurable": {}
    },
    name="conv_agent_assistant"
)
assistant_id = assistant["assistant_id"]
print(f"Assistant created: {assistant_id}")


## Create a LangGraph thread to manage conversation state

Allocating a thread_id to an assistant's run provides it with memory of its
 state between interactions (the list of messages from the user and from the agent).

In [ ]:
# Create a thread to persist conversation state between runs
thread = await client.threads.create()
thread_id = thread["thread_id"]
print(f"Thread created: {thread_id}")

## Begin chatting with the assistant using the thread_id

In [ ]:
user_input = {
    "messages": [
        {
            "role": "user",
            "content": "Salut. Moi c'est Pierre, et toi, qui es-tu?"
        }
    ]
}

async for chunk in client.runs.stream(
    thread_id=thread_id,
    assistant_id=assistant_id,
    input=user_input,
    stream_mode=["updates"]
):
    if chunk.event == "updates":
        print(chunk.data["chatbot"]["messages"][0]["content"])

## Continue chatting, but using LangGraph's streaming built-in feature

In [ ]:
user_input = {
    "messages": [
        {
            "role": "user",
            "content": "Re-dis moi: qui suis-je?."
        }
    ]
}

async for chunk in client.runs.stream(
    thread_id=thread_id,
    assistant_id=assistant_id,
    input=user_input,
    stream_mode=["messages"]
):
    if chunk.event == "messages/partial":
        print(chunk.data[0]["content"])

## Utility streaming function

In [ ]:
async def stream_graph_updates(user_input: str):
    async for chunk in client.runs.stream(
        thread_id=thread_id,
        assistant_id=assistant_id,
        input={"messages": [{"role": "user", "content": user_input}]},
        stream_mode=["updates"]
    ):
        if chunk.event == "updates" and chunk.data.get("chatbot"):
            print(chunk.data["chatbot"]["messages"][0]["content"])


## Chat loop ("quit", "exit" or "q" to break out)

In [ ]:
# Infinite conversational loop
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        await stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        await stream_graph_updates(user_input)
        break

## Check conversational agent with tools graph

In [ ]:
from IPython.display import Image, display # type: ignore
from conv_agent_with_tools import graph

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Connect to the LangGraph server and create an assistant
from langgraph_sdk import get_client # type: ignore

client = get_client(url="http://localhost:2024")

assistant = await client.assistants.create(
    # graph name from langgraph.json
    graph_id="conv_agent_with_tools",
    config={
        "recursion_limit": 1000000000,
        "configurable": {}
    },
    name="conv_agent_assistant"
)
assistant_id = assistant["assistant_id"]
print(f"Assistant created: {assistant_id}")


In [ ]:
# Create a thread to persist conversation state between runs
thread = await client.threads.create()
thread_id = thread["thread_id"]
print(f"Thread created: {thread_id}")

## Begin chatting with the assistant using the thread_id

In [ ]:
# Infinite conversational loop
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        await stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        await stream_graph_updates(user_input)
        break